In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import colorsys

In [ ]:
class Process:
    def __init__(self, pid, arrival_time, burst_time, priority = 0):
        self.pid = pid
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.completion_time = 0
        self.turnaround_time = 0
        self.waiting_time = 0
        self.response_time = -1
        self.processed_time = 0
        self.priority = priority

In [ ]:
processes = [
    Process(1, 1, 6, 3),
    Process(2, 7, 10, 2),
    Process(3, 9, 2, 4),
    Process(4, 25, 3, 1)
]

# ***Algorithms***

### **1.** *FCFS*

In [ ]:
fcfs_chart = []

def FCFS(processes, context_switch):
    time = 0

    processes.sort(key=lambda x: x.arrival_time)
    
    for process in processes:
        if time < process.arrival_time:
            time = process.arrival_time

        if(len(fcfs_chart)):
            time += context_switch / 2
            
        process.waiting_time = time - process.arrival_time
        time += process.burst_time
        process.completion_time = time
        fcfs_chart.append([process.pid, process.completion_time - process.burst_time, process.completion_time])
        process.turnaround_time = process.completion_time - process.arrival_time
        process.response_time = process.waiting_time
        time += context_switch / 2

#### *Result of FCFS*

In [ ]:
def FCFS_Result(processes,context_switch):
    FCFS(processes,context_switch)

    data = {
        'PID': [p.pid for p in processes],
        'Arrival Time': [p.arrival_time for p in processes],
        'Burst Time': [p.burst_time for p in processes],
        'Waiting Time': [p.waiting_time for p in processes],
        'Turnaround Time': [p.turnaround_time for p in processes],
        'Response Time': [p.response_time for p in processes],
        'Completion Time': [p.completion_time for p in processes]
    }

    df = pd.DataFrame(data)
    print(f"Average_WT is: {df['Waiting Time'].mean()}")
    print(f"Average_RT is: {df['Response Time'].mean()}")
    print(f"Average_TT is: {df['Turnaround Time'].mean()}")

    return df

In [ ]:
processes = [
    Process(1, 1, 6),
    Process(2, 7, 10),
    Process(3, 9, 2),
    Process(4, 25, 3)
]
fcfs_df = FCFS_Result(processes,2)
x_max = fcfs_df['Completion Time'].max()

In [ ]:
fcfs_df

In [ ]:
# Gantt chart

fig, ax = plt.subplots(figsize=(10, 6))

for task in fcfs_chart:
    y = task[0]
    start_time = task[1]
    end_time = task[2]
    ax.barh(y, end_time - start_time, left=start_time, height=0.5)

ax.set_xlabel('Time')
ax.set_ylabel('Process')
ax.set_title('FCFS')

# x-axis range
ax.set_xlim(0, x_max)

# Add gridlines for every time slice (every unit on the x-axis)
ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.grid(True, axis='x', linestyle='--', linewidth=0.7)

plt.tight_layout()
plt.show()

### **2.** *SPN*

In [ ]:
spn_chart = []
def SPN(processes, context_switch):
    time = 0
    remaining_processes = processes.copy()
    while remaining_processes:
        ready_queue = [p for p in remaining_processes if p.arrival_time <= time]
        if not ready_queue:
            time += 1
            continue


        ready_queue.sort(key=lambda x: x.burst_time)  # Sort by burst time (shortest first)
        process = ready_queue.pop(0)

        if(len(spn_chart)):
            time += context_switch / 2

        process.waiting_time = time - process.arrival_time
        time += process.burst_time
        process.completion_time = time
        process.turnaround_time = process.completion_time - process.arrival_time
        process.response_time = process.waiting_time
        #print(f"T{time} : P{process.pid} : R{process.completion_time} : Q{len(ready_queue)}")

        remaining_processes.remove(process)
        spn_chart.append([process.pid, process.completion_time - process.burst_time, process.completion_time])
        time += context_switch / 2


#### Result of algorithm

In [ ]:
def SPN_Result(processes,context_switch):
    SPN(processes,context_switch)

    data = {
        'PID': [p.pid for p in processes],
        'Arrival Time': [p.arrival_time for p in processes],
        'Burst Time': [p.burst_time for p in processes],
        'Waiting Time': [p.waiting_time for p in processes],
        'Turnaround Time': [p.turnaround_time for p in processes],
        'Response Time': [p.response_time for p in processes],
        'Completion Time': [p.completion_time for p in processes]
    }

    df = pd.DataFrame(data)

    print(f"Average_WT is: {df['Waiting Time'].mean()}")
    print(f"Average_RT is: {df['Response Time'].mean()}")
    print(f"Average_TT is: {df['Turnaround Time'].mean()}")

    return df


In [ ]:
processes = [
    Process(1, 1, 6),
    Process(2, 7, 10),
    Process(3, 9, 2),
    Process(4, 25, 3)
]

df_spn = SPN_Result(processes,2)
x_max = df_spn['Completion Time'].max()

In [ ]:
df_spn

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for task in spn_chart:
    y = task[0]
    start_time = task[1]
    end_time = task[2]
    ax.barh(y, end_time - start_time, left=start_time, height=0.5)

ax.set_xlabel('Time')
ax.set_ylabel('Process')
ax.set_title('SPN')

ax.set_xlim(0, x_max)

ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.grid(True, axis='x', linestyle='--', linewidth=0.7)

plt.tight_layout()
plt.show()

### **3.** *HRRN*

In [ ]:
hrrn_chart = []

def HRRN(processes, context_switch):
    time = 0
    remaining_processes = processes.copy()
    while remaining_processes:
        ready_queue = [p for p in remaining_processes if p.arrival_time <= time]
        if not ready_queue:
            time += 1
            continue
        for p in ready_queue:
            p.waiting_time = time - p.arrival_time
            p.response_ratio = p.waiting_time / p.burst_time

        ready_queue.sort(key=lambda x: x.response_ratio, reverse=True)
        process = ready_queue.pop(0)

        if(len(hrrn_chart)):
            time += context_switch / 2

        process.waiting_time = time - process.arrival_time
        time += process.burst_time
        process.completion_time = time
        process.turnaround_time = process.completion_time - process.arrival_time
        process.response_time = process.waiting_time

        remaining_processes.remove(process)
        hrrn_chart.append([process.pid, process.completion_time - process.burst_time, process.completion_time])
        # print(f"T{time} : P{process.pid} : R{process.remaining_time} : Q{len(ready_queue)}")
        time += context_switch / 2

#### Result of algorithm

In [ ]:
def HRRN_Result(processes, context_switch):
    HRRN(processes, context_switch)

    data = {
        'PID': [p.pid for p in processes],
        'Arrival Time': [p.arrival_time for p in processes],
        'Burst Time': [p.burst_time for p in processes],
        'Waiting Time': [p.waiting_time for p in processes],
        'Turnaround Time': [p.turnaround_time for p in processes],
        'Response Time': [p.response_time for p in processes],
        'Completion Time': [p.completion_time for p in processes]
    }

    df = pd.DataFrame(data)

    print(f"Average_WT is: {df['Waiting Time'].mean()}")
    print(f"Average_RT is: {df['Response Time'].mean()}")
    print(f"Average_TT is: {df['Turnaround Time'].mean()}")

    return df

In [ ]:
df_hrrn = HRRN_Result(processes, 2)
x_max = df_hrrn['Completion Time'].max()

In [ ]:
df_hrrn

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for task in hrrn_chart:
    y = task[0]
    start_time = task[1]
    end_time = task[2]
    ax.barh(y, end_time - start_time, left=start_time, height=0.5)

ax.set_xlabel('Time')
ax.set_ylabel('Process')
ax.set_title('HRRN')

ax.set_xlim(0, x_max)

ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.grid(True, axis='x', linestyle='--', linewidth=0.7)

plt.tight_layout()
plt.show()

## ***preemptive***

### **4.** *RR*

In [ ]:
rr_chart = []
def RR(processes, quantum=5, context_switch=2):
    time = 0
    remaining_processes = processes.copy()
    queue = []
    temp_q = None

    while remaining_processes or queue or temp_q:

        queue += [p for p in remaining_processes if p.arrival_time <= time]
        remaining_processes = [p for p in remaining_processes if p.arrival_time > time]
        
        if temp_q :
          queue.append(temp_q)
          temp_q = None

        if(len(rr_chart)):
            time += context_switch / 2
            
        if queue:
            
            process = queue.pop(0)
         
            if process.response_time == -1:
                process.response_time = time - process.arrival_time 
            
            if process.remaining_time > quantum:
                time += quantum + (context_switch / 2)
                rr_chart.append([process.pid, time-quantum-(context_switch/2), time-(context_switch/2)])
                process.remaining_time -= quantum
                temp_q = process  # Re-add process to the end of queue if not finished (as temp_q)
            
            else:
                time += process.remaining_time + (context_switch / 2) 
                rr_chart.append([process.pid, time - process.remaining_time-(context_switch/2), time-(context_switch/2)])
                process.remaining_time = 0
                process.completion_time = time - (context_switch / 2)
                process.turnaround_time = process.completion_time - process.arrival_time
                process.waiting_time = process.turnaround_time - process.burst_time
           # print(f"T{time} : P{process.pid} : R{process.remaining_time} : Q{len(queue)} : C{len(remaining_processes)} ")

        else:
            time += 1  

#### Result of algorithm

In [ ]:
def RR_Result(processes):
    RR(processes)

    data = {
        'PID': [p.pid for p in processes],
        'Arrival Time': [p.arrival_time for p in processes],
        'Burst Time': [p.burst_time for p in processes],
        'Waiting Time': [p.waiting_time for p in processes],
        'Turnaround Time': [p.turnaround_time for p in processes],
        'Response Time': [p.response_time for p in processes],
        'Completion Time': [p.completion_time for p in processes]
    }

    df = pd.DataFrame(data)

    print(f"Average_WT is: {df['Waiting Time'].mean()}")
    print(f"Average_RT is: {df['Response Time'].mean()}")
    print(f"Average_TT is: {df['Turnaround Time'].mean()}")

    return df

In [ ]:
processes = [
    Process(1, 1, 6),
    Process(2, 7, 10),
    Process(3, 9, 2),
    Process(4, 25, 3)
]

df_rr = RR_Result(processes)
x_max = df_rr['Completion Time'].max()
y_max = df_rr['PID'].max()

In [ ]:
df_rr

In [ ]:
def get_process_color(pid, total_processes):
    golden_ratio = (1 + 5 ** 0.5) / 2
    hue = (pid * golden_ratio) % 1
    return colorsys.hsv_to_rgb(hue, 0.8, 0.9)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

for task in rr_chart:
    y = task[0]
    start_time = task[1]
    end_time = task[2]
    color = get_process_color(y, y_max)
    ax.barh(y, end_time - start_time, left=start_time, height=0.5, color=color)

ax.set_xlabel('Time')
ax.set_ylabel('Process')
ax.set_title('RR')

ax.set_xlim(0, x_max)

ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.grid(True, axis='x', linestyle=':', linewidth=0.7)

plt.tight_layout()
plt.show()

### **5.** *SRTF*

In [ ]:
srtf_chart = []
def SRTF(processes, quantum, context_switch):
    time = 0
    remaining_processes = processes.copy()
    queue = []
    completed_processes = []
    cs = False

    while len(completed_processes) < len(processes):
        # Add new processes that have arrived at this time
        for p in remaining_processes:
            if p.arrival_time <= time and p not in queue and p not in completed_processes:
                queue.append(p)

        if queue:
            # Sort the queue by remaining burst time (smallest first)
            oldprocess = queue[0]
            queue.sort(key=lambda p: p.remaining_time)
            process = queue[0]
            
            if cs:
                      time += context_switch / 2
                      oldprocess.processed_time = 0
                      cs = False
                      #print("CS1")
            
            if process.response_time == -1:
                process.response_time = time - process.arrival_time  # First time the process is executed
            process.remaining_time -= 1  
            process.processed_time += 1

            #print(f"process {process.pid} Excuted for 1ms.")
            #print(f"remaining_time {process.remaining_time}")
            time += 1
            #print(f"T{time} : P{process.pid} : R{process.remaining_time} : Q{len(queue)} : C{len(completed_processes)} : PT{process.processed_time}")
            srtf_chart.append([process.pid, time-1, time])

            if process.remaining_time == 0:
                #print(f"process {process.pid} completed.")
                process.completion_time = time
                process.turnaround_time = process.completion_time - process.arrival_time
                process.waiting_time = process.turnaround_time - process.burst_time
                completed_processes.append(process)
                queue.remove(process)
                time += context_switch / 2
                cs = True
                #print("CS2")

            if process.processed_time % quantum == 0:
              if process.remaining_time != 0:
                 time += context_switch / 2
                 cs = True

        else:
            time += 1
            #print("D")


    return completed_processes

#### Result of algorithm

In [ ]:
def SRTF_Result(processes, quantum, context_switch):
    completed_processes = SRTF(processes, quantum, context_switch)

    data = {
        'PID': [p.pid for p in completed_processes],
        'Arrival Time': [p.arrival_time for p in completed_processes],
        'Burst Time': [p.burst_time for p in completed_processes],
        'Completion Time': [p.completion_time for p in completed_processes],
        'Turnaround Time': [p.turnaround_time for p in completed_processes],
        'Waiting Time': [p.waiting_time for p in completed_processes],
        'Response Time': [p.response_time for p in completed_processes],
    }

    df = pd.DataFrame(data)

    print(f"Average_WT is: {df['Waiting Time'].mean()}")
    print(f"Average_RT is: {df['Response Time'].mean()}")
    print(f"Average_TT is: {df['Turnaround Time'].mean()}")


    return df

In [ ]:
processes = [
    Process(1, 1, 6),
    Process(2, 7, 10),
    Process(3, 9, 2),
    Process(4, 25, 3)
]
srtf_df = SRTF_Result(processes, 5, 2)
x_max = srtf_df['Completion Time'].max()
y_max = srtf_df['PID'].max()

In [ ]:
srtf_df

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

for task in srtf_chart:
    y = task[0]
    start_time = task[1]
    end_time = task[2]
    color = get_process_color(y, y_max)
    ax.barh(y, end_time - start_time, left=start_time, height=0.5, color=color)

ax.set_xlabel('Time')
ax.set_ylabel('Process')
ax.set_title('SRTF')

ax.set_xlim(0, x_max)

ax.xaxis.set_major_locator(plt.MultipleLocator(1))
ax.grid(True, axis='x', linestyle='--', linewidth=0.7)

plt.tight_layout()
plt.show()

# ***Run All***

In [ ]:
# Function to run all algorithms and collect results

def run_all_algorithms(processes):
    processes = [
    Process(1, 1, 6),
    Process(2, 7, 10),
    Process(3, 9, 2),
    Process(4, 25, 3)
]


    results = []
    algorithms = ['FCFS', 'SPN', 'HRRN', 'RR', 'SRTF']

    for algorithm in algorithms:
        processes_copy = [Process(p.pid, p.arrival_time, p.burst_time) for p in processes]

        if algorithm == 'FCFS':
            FCFS(processes_copy, context_switch=2)
        elif algorithm == 'SPN':
            SPN(processes_copy, context_switch=2)
        elif algorithm == 'HRRN':
            HRRN(processes_copy, context_switch=2)
        elif algorithm == 'RR':
            RR(processes_copy, quantum=5, context_switch=2)
        elif algorithm == 'SRTF':
            SRTF(processes_copy, quantum=5, context_switch=2)


        for p in processes_copy:
            results.append({
                'Algorithm': algorithm,
                'PID': p.pid,
                'Arrival Time': p.arrival_time,
                'Burst Time': p.burst_time,
                'Waiting Time': p.waiting_time,
                'Response Time': p.response_time,
                'Turnaround Time': p.turnaround_time,
                'Completion Time': p.completion_time
            })


    df = pd.DataFrame(results)
    return df

In [ ]:
df_results = run_all_algorithms(processes)
df_results

In [ ]:
avg_df = df_results[['Algorithm', 'Waiting Time', 'Response Time', 'Turnaround Time']].groupby('Algorithm').mean()

In [ ]:
# Resetting the index to make 'Algorithm' a column
avg_df.reset_index(inplace=True)

#changing the name of columns
avg_df.columns = ['Algorithm', 'AWT', 'ART', 'ATT']

avg_df